In [ ]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'Enter your key'

In [ ]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500) 

In [ ]:
#pip install openai

In [ ]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

In [ ]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
import dill

# Configure dill to handle SSLContext
dill.settings['recurse'] = True

file_path = "vector_index.pkl"

# Extract only the FAISS index to avoid SSLContext errors
faiss_index = vectorindex_openai.index  # FAISS index object (without retriever)

with open(file_path, "wb") as f:
    dill.dump((faiss_index, vectorindex_openai.docstore, vectorindex_openai.index_to_docstore_id), f)

In [ ]:
# Load FAISS index 
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        faiss_index, docstore, index_to_docstore_id = dill.load(f)

# Reconstruct FAISS vector store
vectorIndex = FAISS(
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=None  # Set this if needed
)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain